bert_naver_movie_colab_ipynb(https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x) 참고

## 1.Colab 환경 설정

In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [35]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2.데이터셋 불러오기

In [37]:
import csv
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/TrainSet _1st.csv',encoding="utf-8")
data.columns=['Findings','Conclusion','AcuteInfarction']

#findings 데이터와 Conclusion 데이터를 하나의 column에 합치기
for i in range(len(data)):
    data.Findings[i] = str(data.Findings[i]) + ' ' + str(data.Conclusion[i])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
data.shape

(6190, 3)

In [39]:
data.sample(n=5)

,Findings,Conclusion,AcuteInfarction
715,Clinical information : Lung cancer \r\n\r\nAxi...,No definite evidence of brain metastasis.\n,0
6081,"Clinical information : ICA stenosis, proximal\...",1. Diffuse brain atrophy.\r\n2. Microangiopath...,0
1299,nan mild diffuse T2 hyperintensities at the bi...,mild diffuse T2 hyperintensities at the bilate...,0
1217,"Clinical information : \r\n\r\nAxial T1WI, sag...",Normal brain MRI.\n,0
2179,"Clinical information : Headache\n\nAxial T1WI,...",1. No evidence of acute infarction.\r\n2. Micr...,0


In [40]:
data_shuffled = data.sample(frac=1).reset_index(drop=True) #데이터 랜덤으로 셔플

In [41]:
#train data & test data 설정
x = len(data) #0 ~ x 까지 학습 데이터, x~끝 까지 테스트 데이터 
train = data_shuffled[:x]
test = data_shuffled[x:]


(6190, 3)
(0, 3)


In [42]:
print(train.shape)
print(test.shape)

display(train.head())
display(test.head())

,Findings,Conclusion,AcuteInfarction
0,Clinical information : ung cancer\r\n\r\nAxial...,1. No significant change of \r\n - indetermina...,0
1,"Clinical information : \r\n\r\nAxial T1WI, sag...",Normal brain MRI.\n,0
2,nan Left proximal ICA occlusion.\r\nFocal sten...,Left proximal ICA occlusion.\r\nFocal stenoses...,0
3,* Follow-up for cerebral metastasis.\r\n\r\nAx...,No significant change in size of multiple meta...,0
4,"Clinical information : Disorientation, unspeci...","1. Acute infarction in left parietal lobe, tem...",1


,Findings,Conclusion,AcuteInfarction


## 3.Train set 전처리

In [43]:
# CLS, SEP 붙이기 (문장의 시작, 끝 구분)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Findings]

In [ ]:
labels = train['AcuteInfarction'].values #결과 라벨 할당

In [ ]:
import pandas as pd
from transformers import BertTokenizer


In [47]:
#사전학습모델 bert-base-multilingual-cased 사용, License: Apache 2.0
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences] #토큰화

In [48]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] Clinical information : ung cancer

Axial T1WI, sagittal T1WI, axial T2WI, axial FLAIR, axial T2* GRE image 획득하였으며 조영증강을 시행함.
 1. No significant change of 
 - indeterminate focal enhancing lesion at right parietal bone
 - focal enhancing lesion, probable LN, at both posterior neck space 
2. Slightly decreased size of enhancing lesions at left parotid gland
3. Microangiopathy

 [SEP]
['[CLS]', 'Clinical', 'information', ':', 'ung', 'cancer', 'A', '##xia', '##l', 'T1', '##W', '##I', ',', 'sa', '##gitt', '##al', 'T1', '##W', '##I', ',', 'a', '##xia', '##l', 'T2', '##W', '##I', ',', 'a', '##xia', '##l', 'FL', '##A', '##IR', ',', 'a', '##xia', '##l', 'T2', '*', 'GR', '##E', 'image', '획', '##득', '##하였으며', '조', '##영', '##증', '##강', '##을', '시', '##행', '##함', '.', '1', '.', 'No', 'significant', 'change', 'of', '-', 'inde', '##ter', '##minat', '##e', 'focal', 'en', '##han', '##cing', 'les', '##ion', 'at', 'right', 'pari', '##eta', '##l', 'bone', '-', 'focal', 'en', '##han', '##cing', 'les',

In [49]:
MAX_LEN = 509 #최대 시퀀스 길이, bert모델의 인식 가능한 최대 길이가 512, 따라서 CLS와 SEP를 빼고 510까지 할당 가능
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [50]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [51]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [52]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [53]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.2)

In [54]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.2)

In [55]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [56]:
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## 4.Test set 전처리

In [ ]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.Findings]

# 라벨 데이터
labels = test['AcuteInfarction'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 600
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 16
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 5.모델 생성

In [57]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [58]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

In [60]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## 6.모델 학습

In [61]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [62]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습 # 배치 사이즈 = 16, epoch = 4 
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음 #배치사이즈 16
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids,token_type_ids=None,
                        attention_mask=b_input_mask,labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.13
  Training epcoh took: 0:07:48

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:45

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.05
  Training epcoh took: 0:07:53

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:45

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:07:53

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:45

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.02
  Training epcoh took: 0:07:53

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:45

Training complete!


In [64]:
model.save_pretrained('/content/drive/MyDrive/Outputs_model(100)') #모델저장
tokenizer.save_pretrained('/content/drive/MyDrive/Outputs_model(100)') #토크나이저 저장

('/content/drive/MyDrive/Outputs_model(100)/tokenizer_config.json',
 '/content/drive/MyDrive/Outputs_model(100)/special_tokens_map.json',
 '/content/drive/MyDrive/Outputs_model(100)/vocab.txt',
 '/content/drive/MyDrive/Outputs_model(100)/added_tokens.json')

## 7.테스트셋 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))